# SD214, TP3 : Text segmentation using hidden Markov model

Raphaël Teboul & Félix Gaschi

In [1]:
import numpy as np
import subprocess

## Question 1

At the beggining, the decoding starts in the state 1. So $\pi = \begin{pmatrix}
1 \\
0
\end{pmatrix}$

## Question 2

The probability to move from state 1 to state 2 is the probability $P(2|1)$ to be at state 2 knowing that we were at state 1 on the previous iteration of the Markov chain. So it's given by $A(1, 2) = 0.00078192196418794$

The probability to remain in state 2 is $A(2,2) = 1$. It means that once in state 2, we remain in state 2. Indeed, once we are in the body of the mail, we won't find the header again. 

The higher probability is the probability to remain in state 2 and the lower probability is (accordingly) the probability to move from state 2 to state 1. Indeed these probabilities are complementary, and as we said earlier, once we reached the body of the mail, we won't find ourselves in the header again. So the probability to move from state 2 to 1 is 0.

Remark : With this simple model, we can easily describe the evolution of the probability to be in 1 or 2 at the n-th iteration. we have:

$$
\pi(n) = \begin{pmatrix}
A(1,1)^n \\
1 - A(1,1)^n 
\end{pmatrix}
$$

So first, $\pi(n)_1 > \pi(n)_2$. And then it switches. Let's calculate $n$ such that the probability order switches :
$A(1,1)^n = \frac{1}{2} \Leftrightarrow n = \frac{- \log 2}{\log A(1,1)}$

In [2]:
n = (- np.log(2)) / np.log(0.999218078035812)

print("n = ", np.ceil(n))

n =  887.0


Assuming that the initial probality is at $n=0$, we have $\pi(n)_1 > \pi(n_2)$ for $n < 887$ and then $\pi(n)_2 > \pi(n)$ for $n \geq 887$

## Question 3

The corresponding matrix is $B$ of size $(2, N)$ such that $B(s, c) = P(c|s)$

## Viterbi implementation

In [3]:
def removeZeros(A):
    A[A == 0.] = np.finfo(np.double).tiny
    return A
    
def viterbi(y, A, B, pi):
    """input:
    - y : sequence of observations (int array)
    - A : transition matrix (double array)
    - B : emission matrix (double array)
    - pi: initial vector (double array)
    output:
    sequence of states"""
    
    k = A.shape[0]
    N = B.shape[1]
    m = len(y)
    
    T1 = np.zeros((k, m))
    T2 = np.zeros((k, m))
    
    A = removeZeros(A)
    B = removeZeros(B)
    pi = removeZeros(pi)
    Alog = np.log(A)
    Blog = np.log(B)
    pilog = np.log(pi)
    
    T1[:,0] = pilog + Blog[:, y[0]]
    
    for i in range(1, m):
        for j in range(k):
            vect = T1[:, i-1] + Alog[:,j] + Blog[j, y[i]]
            T1[j, i] = np.max(vect)
            T2[j, i] = np.argmax(vect)
            
    x = np.zeros((m,))
    
    x[m-1] = np.argmax(T1[:,m-1])
    
    for i in range(m-1, 0, -1):
        x[i-1] = T2[int(x[i]), i]
    
    return x
    
        
        

In [4]:
def getObs(seq):
    res = []
    for i in seq:
        fileName = "dat/mail" + str(i) + ".dat"
        y = []
        with open(fileName, "r") as f:
            for line in f:
                y += [int(line)]
        res += [y]
    return res

A = np.array([[0.99921878035812, 0.000781921964187974], [0,1]])

fileName = "P.text"
B = np.loadtxt(fileName).T

pi = np.array([1.,0.])

## Question 4

In [5]:
seq = getObs(range(11,30))

In [6]:
paths = []

for obs in seq:
    paths += [viterbi(obs, A, B, pi)]


In [7]:
for i, X in enumerate(paths):
    np.savetxt("dat/path" + str(11 + i) + ".dat", X + np.ones(X.shape), fmt="%d", newline="")

In [30]:
import os


res = subprocess.check_output(["perl", "segment.pl", "dat/mail11.txt", "dat/path11.dat"]).decode("utf-8")[len(paths[0]):]


In [31]:
print(res)

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

For the email 11, we see that the method has worked

In [32]:
print(subprocess.check_output(["perl", "segment.pl", "dat/mail15.txt", "dat/path15.dat"]).decode("utf-8")[len(paths[4]):])

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

For the email 15, it's almost at the right place. The line with "-" and the web link may have delayed the moment when we switch state. 

In [33]:
print(subprocess.check_output(["perl", "segment.pl", "dat/mail26.txt", "dat/path26.dat"]).decode("utf-8")[len(paths[15]):])

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

For the email 26, the portions of mails included in the body have not perturbed the result. (If we assume that they must be in the body)

In [34]:
print(subprocess.check_output(["perl", "segment.pl", "dat/mail14.txt", "dat/path14.dat"]).decode("utf-8")[len(paths[3]):])

1111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111

For the email 14 however, we see that the result seem to be altered by the presence of portions of other mails inside the body. And the header of these portion and the ">" might change the characters distribution of the body and make it closer to the typical characters distribution of a header. 

**Conclusion**: The method is working quite well. However, the presence of links, and mostly part of other mails in the body can alter the result by changing the characters distribution.